In [ ]:
import os
import pandas as pd
from connections import * 
import boto3
from datetime import datetime as dt

In [ ]:
"""
Aqui faço um for para listar os arquivos recebidos e carregar os seus nomes em uma lista
"""
for _,_, files in os.walk(conns_paths.sources):
    pass

In [ ]:
'''
Nesta célula há a iteração da lista de nomes dos arquivos visando carregá-los para dataframe
'''
files_list = list()
for f in files:    
    df = 'df_'+str(f)[:str(f).find('.')].strip("[|'|\"")
    files_list.append(df)
    prog = f"{df} = pd.read_csv('{conns_paths.sources}/{f}',sep=';',encoding='utf8')"
    exec(prog)

In [ ]:
'''Estas linhas comentadas foram usadas para reconhecimento das colunas disponíveis em cada arquivo
para que fosse feito o merge necessário entre os dataframes.
'''
# Merge this on a Dimtable apart!! df_categories_columns = ['category_id', 'category_name', 'description', 'picture']
# df_customers_columns = ['customer_id', 'company_name', 'contact_name', 'contact_title','address', 'city', 'region', 'postal_code', 'country', 'phone', 'fax']
# df_employees_columns = ['employee_id', 'last_name', 'first_name', 'title', 'title_of_courtesy','birth_date', 'hire_date', 'address', 'city', 'region', 'postal_code','country', 'home_phone', 'extension', 'photo', 'notes', 'reports_to','photo_path']
# Dont merge this on Fact!!! df_employee_territories_columns = ['employee_id', 'territory_id']
# df_orders.columns = ['order_id', 'customer_id', 'employee_id', 'order_date', 'required_date','shipped_date', 'ship_via', 'freight', 'ship_name', 'ship_address','ship_city', 'ship_region', 'ship_postal_code', 'ship_country']
# df_order_details_columns  = ['order_id', 'product_id', 'unit_price', 'quantity', 'discount']
# df_products_columns = ['product_id', 'product_name', 'supplier_id', 'category_id','quantity_per_unit', 'unit_price', 'units_in_stock', 'units_on_order','reorder_level', 'discontinued']
# Merge this on a table apart!! df_region_columns = ['region_id', 'region_description']
# df_shippers_columns = ['shipper_id', 'company_name', 'phone']
# Merge this on a Dimtable apart!! df_suppliers_columns = ['supplier_id', 'company_name', 'contact_name', 'contact_title','address', 'city', 'region', 'postal_code', 'country', 'phone', 'fax','homepage']
# Merge this on a Dimtable apart!! df_territories_columns = ['territory_id', 'territory_description', 'region_id']
# Merge this on a Dimtable apart!! df_us_states_columns = ['state_id', 'state_name', 'state_abbr', 'state_region']

In [ ]:
'''
Após reconhecimento, pude entender como os dataframes se relacionam, e aqui fiz o merge.
'''

fact = df_orders.merge(df_order_details,left_on='order_id',right_on='order_id',how='left')\
.merge(df_customers,left_on='customer_id',right_on='customer_id',how='left')\
.merge(df_shippers,left_on='ship_via',right_on='shipper_id',how='left')\
.merge(df_employees,left_on='employee_id',right_on='employee_id',how='left')\
.merge(df_products,left_on='product_id',right_on='product_id',how='left')


In [ ]:
'''
Aqui listei os campos, que a primeira vista, julguei desnecessários para a análise.

'''

# FactAvailableColumns = ['order_id', 'customer_id', 'employee_id', 'order_date', 'required_date',
#        'shipped_date', 'ship_via', 'freight', 'ship_name', 'ship_address',
#        'ship_city', 'ship_region', 'ship_postal_code', 'ship_country',
#        'product_id', 'unit_price_x', 'quantity', 'discount', 'company_name_x',
#        'contact_name', 'contact_title', 'address_x', 'city_x', 'region_x',
#        'postal_code_x', 'country_x', 'phone_x', 'fax', 'shipper_id',
#        'company_name_y', 'phone_y', 'last_name', 'first_name', 'title',
#        'title_of_courtesy', 'birth_date', 'hire_date', 'address_y', 'city_y',
#        'region_y', 'postal_code_y', 'country_y', 'home_phone', 'extension',
#        'photo', 'notes', 'reports_to', 'photo_path', 'product_name',
#        'supplier_id', 'category_id', 'quantity_per_unit', 'unit_price_y',
#        'units_in_stock', 'units_on_order', 'reorder_level', 'discontinued']

columns_to_drop = ['contact_name', 'contact_title', 'address_x','postal_code_x','phone_y', 'last_name', 'first_name', 'title','title_of_courtesy','birth_date','address_y', 'city_y','postal_code_y', 'country_y', 'home_phone', 'extension','photo', 'notes','photo_path']

In [ ]:
'''
Aqui retirei as colunas e ordenei o dataframe por customer id e order date
'''
fact = fact.drop(columns=columns_to_drop)
fact = fact.sort_values(['customer_id','order_date'])

In [ ]:
'''Aqui carrego o dataframe com as colunas necessarias para csv em pasta local'''
fact.to_csv(conns_paths.datalakesources+'/Fact.csv',sep=',',index=False)

In [ ]:
'''
Nesta célula carrego o dataframe para o S3
'''
for _,_, datalakefiles in os.walk(conns_paths.datalakesources):
    pass


# aqui falta terminar for para carregar o s3
s3_client = boto3.client('s3')
# "s3_client.upload_file(f'{conns_paths.datalakesources}/{dlf}',{connections.bucket},f/"landing/{dlf}\")"

for dlf in datalakefiles:    
    load_dl = f's3_client.upload_file("{conns_paths.datalakesources}/{dlf}","{conns_paths.bucket}","{conns_paths.landing}{dlf}")'
    exec(load_dl)